In [8]:
#Single Machine Scheduling

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [9]:
single_mach = pd.DataFrame({"Arrival Time" : [0,0,0,0,0,0,0,0,0,0],
                            'Job Number':[1,2,3,4,5,6,7,8,9,10], 
                            'Processing Time':[8,2,2,8,8,4,6,9,5,10], 
                            'Due Date':[4,13,2,8,5,17,6,19,7,8]})
single_mach

,Arrival Time,Job Number,Processing Time,Due Date
0,0,1,8,4
1,0,2,2,13
2,0,3,2,2
3,0,4,8,8
4,0,5,8,5
5,0,6,4,17
6,0,7,6,6
7,0,8,9,19
8,0,9,5,7
9,0,10,10,8


In [10]:
#Order according to SPT
single_mach = single_mach.sort_values('Processing Time')
single_mach['Completion Time'] = single_mach['Processing Time'].cumsum()
single_mach['Tardiness'] = np.maximum(0, single_mach['Completion Time'] - single_mach['Due Date'])
single_mach["Lateness"] = single_mach["Completion Time"] - single_mach["Due Date"]


single_mach

,Arrival Time,Job Number,Processing Time,Due Date,Completion Time,Tardiness,Lateness
1,0,2,2,13,2,0,-11
2,0,3,2,2,4,2,2
5,0,6,4,17,8,0,-9
8,0,9,5,7,13,6,6
6,0,7,6,6,19,13,13
0,0,1,8,4,27,23,23
3,0,4,8,8,35,27,27
4,0,5,8,5,43,38,38
7,0,8,9,19,52,33,33
9,0,10,10,8,62,54,54


In [11]:
import random
# create random df following this code: l = [random.randint(0,10) for i in range(5)]

random_jobs = pd.DataFrame({"Arrival Time":[40 + random.randint(10,40) for i in range(10)],
                            'Job Number' : [11,12,13,14,15,16,17,18,19,20],
                            'Processing Time':[random.randint(1,10) for i in range(10)], 
                            'Due Date':[7 + random.randint(0,25) for i in range(10)],
                             "Completion Time" : [0,0,0,0,0,0,0,0,0,0],
                             "Tardiness" : [0,0,0,0,0,0,0,0,0,0],
                             "Lateness" : [0,0,0,0,0,0,0,0,0,0]})
random_jobs = random_jobs.sort_values('Arrival Time')


random_jobs

,Arrival Time,Job Number,Processing Time,Due Date,Completion Time,Tardiness,Lateness
4,53,15,6,23,0,0,0
7,56,18,9,21,0,0,0
0,65,11,7,31,0,0,0
5,65,16,10,30,0,0,0
8,66,19,2,25,0,0,0
9,66,20,10,30,0,0,0
1,77,12,9,29,0,0,0
6,79,17,3,28,0,0,0
2,80,13,6,15,0,0,0
3,80,14,6,31,0,0,0


In [12]:
# Optimizing for SPT 

# Step 1. Create 2 data frames to append and drop data

pending_jobs = pd.DataFrame()
reschedule = pd.DataFrame()

while random_jobs[random_jobs['Processing Time'] > 0].shape[0] > 0:   

    # Step 2. Sort random jobs by Arrival Time and compare Arrival Time from first new random job vs. Completion Time of original jobs.
    # Identify first original job with a Completion Time > than first new Arrival Time
    random_jobs = random_jobs.sort_values('Arrival Time')

    print(random_jobs.iloc[0,0])
    
    # Step 3. Start using pending jobs df.

    pending_jobs = single_mach.loc[(single_mach["Completion Time"] > random_jobs.iloc[0,0])]
    pending_jobs = pending_jobs.reset_index()

    print(random_jobs.iloc[0])

    pending_jobs.drop(["index"], axis = 1, inplace=True)
    
    # Here we set the Arrival Time of the new pending jobs = Completion Time of the job that got out of single_mach.
    # We also drop the first row because the first task is being processed
    pending_jobs["Arrival Time"] = pending_jobs.iloc[0,4]
    pending_jobs = pending_jobs.drop([0] , axis = 0)
    single_mach = single_mach[~single_mach['Job Number'].isin(pending_jobs['Job Number'])]
    
    print("Pending_jobs")
    print(pending_jobs)
    
    # Send pending jobs back to random jobs.
    # Also empty pending jobs df.
    random_jobs = random_jobs.append(pending_jobs)
    pending_jobs = pending_jobs[0:0]
    
    # Step 4. Start filling reschedule df. 
    # Append data from single_mach. Look for the Arrival time <= to the Completion Time of last job in the df.

    print(single_mach.iloc[-1,4])

    reschedule = random_jobs.loc[(random_jobs["Arrival Time"] <= single_mach.iloc[-1,4])]

    print("Reschedule")
    print(reschedule)

    #Step 5

    reschedule = reschedule.sort_values("Processing Time")

    #Step 6

    random_jobs = random_jobs[~random_jobs['Job Number'].isin(reschedule['Job Number'])]

    random_jobs = random_jobs.sort_values("Arrival Time")
    
    print("Randomjobs al terminar Step 6")
    print(random_jobs)

    #Step 7

    single_mach = single_mach.append(reschedule)

    reschedule = reschedule[~reschedule['Job Number'].isin(single_mach['Job Number'])]

    print("Reschedule")
    print(reschedule)

    #Recompute Completion Time

    single_mach['Completion Time'] = single_mach['Processing Time'].cumsum()
    single_mach['Tardiness'] = np.maximum(0, single_mach['Completion Time'] - single_mach['Due Date'])
    single_mach["Lateness"] = single_mach["Completion Time"] - single_mach["Due Date"]

    print("Single_mach")
    print(single_mach)

53
Arrival Time       53
Job Number         15
Processing Time     6
Due Date           23
Completion Time     0
Tardiness           0
Lateness            0
Name: 4, dtype: int64
Pending_jobs
Empty DataFrame
Columns: [Arrival Time, Job Number, Processing Time, Due Date, Completion Time, Tardiness, Lateness]
Index: []
62
Reschedule
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
4            53          15                6        23                0   
7            56          18                9        21                0   

   Tardiness  Lateness  
4          0         0  
7          0         0  
Randomjobs al terminar Step 6
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
0            65          11                7        31                0   
5            65          16               10        30                0   
8            66          19                2        25                0   
9            66          20         

In [13]:
single_mach

,Arrival Time,Job Number,Processing Time,Due Date,Completion Time,Tardiness,Lateness
1,0,2,2,13,2,0,-11
2,0,3,2,2,4,2,2
5,0,6,4,17,8,0,-9
8,0,9,5,7,13,6,6
6,0,7,6,6,19,13,13
0,0,1,8,4,27,23,23
3,0,4,8,8,35,27,27
4,0,5,8,5,43,38,38
7,0,8,9,19,52,33,33
9,0,10,10,8,62,54,54


In [14]:
#Performance Metrics with SPT

f_bar = single_mach["Completion Time"].mean()
t_bar = single_mach["Processing Time"].mean()
mean_waiting_time = f_bar-t_bar

t_max = single_mach["Tardiness"].max()

pos_count = [num for num in single_mach["Lateness"] if num >= 1] 
late_jobs = len(pos_count)

print("Mean Waiting time = "+ str(mean_waiting_time))
print("T Max = "+ str(t_max))
print("Number of Late Jobs = "+ str(late_jobs))

Mean Waiting time = 54.05
T Max = 100
Number of Late Jobs = 18
